In [1]:
import random

from deap import base
from deap import creator
from deap import tools

In [2]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

IND_SIZE = 8 * 14

INT_MIN, INT_MAX = 0, 9
toolbox = base.Toolbox()
toolbox.register("attr_int", random.randint, INT_MIN, INT_MAX)
toolbox.register("individual", tools.initRepeat, creator.Individual, 
                 toolbox.attr_int, IND_SIZE)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [3]:
def evalOneMax(individual):
    return sum(individual),

In [4]:
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=9, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

In [5]:
def main():
    pop = toolbox.population(n=300)
    
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    CXPB, MUTPB = 0.5, 0.2
    fits = [ind.fitness.values[0] for ind in pop]
    g = 0

    while max(fits) < 1000 and g < 1000:
        g = g + 1
    
         # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
        
        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values
        
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        pop[:] = offspring
        
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        if g % 100 == 0:
            print("-- Generation %i --" % g)
            length = len(pop)
            mean = sum(fits) / length
            sum2 = sum(x*x for x in fits)
            std = abs(sum2 / length - mean**2)**0.5

            print("  Min %s" % min(fits))
            print("  Max %s" % max(fits))
            print("  Avg %s" % mean)
            print("  Std %s" % std)

    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s %s" % (best_ind, best_ind.fitness.values))
    

In [7]:
if __name__ == "__main__":
    main()

-- Generation 100 --
  Min 879.0
  Max 939.0
  Avg 933.06
  Std 10.174956183364497
-- Generation 200 --
  Min 908.0
  Max 961.0
  Avg 955.2733333333333
  Std 10.484208230584862
-- Generation 300 --
  Min 917.0
  Max 977.0
  Avg 970.7866666666666
  Std 10.988228044393153
-- Generation 400 --
  Min 891.0
  Max 992.0
  Avg 985.3533333333334
  Std 13.624554630843814
-- Generation 500 --
  Min 943.0
  Max 995.0
  Avg 989.81
  Std 10.910571326326897
-- Generation 600 --
  Min 941.0
  Max 999.0
  Avg 992.9466666666667
  Std 11.631730548609442
Best individual is [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 9, 9, 9, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 9, 9, 8, 9, 9, 9] (1000.0,)
